In [ ]:
import shutil
import numpy as np
import yaml
import pandas as pd
import sys
import re
import shlex, subprocess, os

## Reading config and file names

In [ ]:
os.chdir((sys.path[0]))

In [ ]:
with open("../configs/config.yaml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)
    print("Read successful")
print(config)

In [ ]:
raw = config['data']['raw']
fastp = config['data']['fastp']
fastp_json = config['data']['fastp_json']

In [ ]:
metadata = pd.read_csv("../metadata/drugs_metadata.tsv", sep="\t")
mapping = dict(zip(metadata.iloc[:, 1], metadata.iloc[:, 0]))
mapping = {
    re.sub(r'\.fastq(?:\.gz)?$|\.fq(?:\.gz)?$', '', k): v
    for k, v in mapping.items()
}
with open("../configs/multiqc_config.yaml", "w") as f:
    yaml.dump({"sample_names_replace": mapping}, f, default_flow_style=False)

In [ ]:
os.chdir(raw)
files = [f for f in os.listdir('.') if os.path.isfile(f)]
files.sort()
len(files)

In [ ]:
ids = np.unique([
    "_".join(f.split("_")[2:5])
    for f in files
])
len(ids)

## Perform quality control with fastp

In [ ]:
for i in range(len(ids)):
    path = "".join([fastp, '/',ids[i],"_fastp"])
    r1 = files[i*2]
    r2 = files[i*2+1]
    out = "".join([path, "/", ids[i]])
    print(out)
    
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)
        os.makedirs(path)

    os.system(f"fastp -i {r1} -I {r2} -h {out}.html -j {out}.json -o {out}\_R1\_fastp.fq.gz -O {out}\_R2\_fastp.fq.gz -w 16")

## Assemble results with multiqc

In [ ]:
os.chdir(fastp)
dirs = os.listdir(".")
len(dirs)

In [ ]:
os.system(f"find . -name *.json -exec cp {{}} {fastp_json} \;")

In [ ]:
os.chdir(fastp_json)
os.system("for filename in *.json; do mv $filename ${filename%.*}_fastp.json; done;")

In [ ]:
files = os.listdir(".")
len(files)

In [ ]:
os.system(f"multiqc {fastp_json} --filename multiqc_report_fastp -o {fastp_json}/.. -f --interactive -c ../configs/multiqc_config.yaml")

## STAR index

In [ ]:
config["star"]

In [ ]:
genome_keys = {
    "genomeDir": "--genomeDir",
    "genomeFastaFiles": "--genomeFastaFiles",
    "sjdbGTFfile": "--sjdbGTFfile",
    "sjdbOverhang": "--sjdbOverhang",
    "runThreadN": "--runThreadN",
    "genomeSAindexNbases": "--genomeSAindexNbases",
    "genomeChrBinNbits": "--genomeChrBinNbits",
    "genomeSAsparseD": "--genomeSAsparseD",
    "sjdbGTFfeatureExon": "--sjdbGTFfeatureExon",
    "sjdbGTFtagExonParentTranscript": "--sjdbGTFtagExonParentTranscript",
    "sjdbGTFtagExonParentGene": "--sjdbGTFtagExonParentGene",
    "sjdbInsertSave": "--sjdbInsertSave",
}

cmd = ["STAR", "--runMode", "genomeGenerate"]
for k, flag in genome_keys.items():
    if k in config["star"] and config["star"][k] not in (None, ""):
        v = config["star"][k]
        if k == "genomeFastaFiles" and isinstance(v, list):
            v = " ".join(map(str, v))
        cmd += [flag, str(v)]

In [ ]:
required = ["genomeDir", "genomeFastaFiles"]
for r in required:
    if r not in config["star"]:
        raise ValueError(f"Missing required parameter for genomeGenerate: {r}")

# Make genomeDir if needed
os.makedirs(config["star"]["genomeDir"], exist_ok=True)

print("Will run:\n", " ".join(shlex.quote(c) for c in cmd))

In [ ]:
os.system(" ".join(shlex.quote(c) for c in cmd))

## Star alignment

In [ ]:
align_flag_map = {
    "runThreadN": "--runThreadN",
    "genomeDir": "--genomeDir",
    "outFilterType": "--outFilterType",
    "outFilterMultimapNmax": "--outFilterMultimapNmax",
    "alignSJoverhangMin": "--alignSJoverhangMin",
    "alignSJDBoverhangMin": "--alignSJDBoverhangMin",
    "outFilterMismatchNmax": "--outFilterMismatchNmax",
    "outFilterMismatchNoverLmax": "--outFilterMismatchNoverLmax",
    "alignIntronMin": "--alignIntronMin",
    "alignIntronMax": "--alignIntronMax",
    "alignMatesGapMax": "--alignMatesGapMax",
    "outSAMtype": "--outSAMtype",
    "outBAMcompression": "--outBAMcompression"
}

def build_star_align_cmd(star_params, R1, R2=None, out_prefix="sample", out_dir="."):
    os.makedirs(out_dir, exist_ok=True)
    prefix_path = os.path.join(out_dir, out_prefix) + "."
    
    cmd = ["STAR"]
    for k, flag in align_flag_map.items():
        if k in star_params and star_params[k] not in (None, ""):
            v = star_params[k]
            if isinstance(v, (list, tuple)):
                cmd += [flag] + [str(x) for x in v]
            else:
                # also handle space-separated strings like "BAM SortedByCoordinate"
                parts = str(v).split()
                cmd += [flag] + parts
    
    if R2:
        cmd += ["--readFilesIn", R1, R2]
    else:
        cmd += ["--readFilesIn", R1]
    
    # auto-detect gz
    if R1.endswith(".gz") or (R2 and R2.endswith(".gz")):
        cmd += ["--readFilesCommand", "gunzip", "-c"]
    
    cmd += ["--outFileNamePrefix", prefix_path]
    return cmd

In [ ]:
for i in range(len(ids)):
    path = "".join([fastp, '/',ids[i],"_fastp"])
    r1 = "".join([ids[i],"_R1_fastp.fq.gz"])
    r2 = "".join([ids[i],"_R2_fastp.fq.gz"])
    cmd = build_star_align_cmd(config["star"],
                           "".join([path, "/", r1]), "".join([path, "/", r2]),
                           out_prefix=ids[i],
                           out_dir = "".join([config["data"]["star"], "/", ids[i]]))
    print(cmd)
    subprocess.run(cmd, check=True)